In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
# Do this just once since it's long. Can use the local file afterwards

#!wget 'https://zenodo.org/record/4088982/files/data%20release.csv'

In [3]:
boostr= pd.read_csv("data release.csv")
boostr

,time,file,B:ACMNIG,B:ACMNPG,B:ACMXIG,B:ACMXPG,B:DCIG,B:DCPG,B:GMPS4V,B:GMPSBT,...,B_IMINXG,B_IMINXO,B_IPHSTC,B_VIMAX,B_VIMIN,B_VINHBT,B_VIPHAS,I:IB,I:MDAT40,I:MXIB
0,2020-03-10T05:00:00.081Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,136.134511,0.0003,...,-11.825,0.0,20.0,966.105041,103.393158,1.0,1.81,7055.9375,7033.233614,7033.410645
1,2020-03-10T05:00:00.148Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,123.134498,0.0003,...,-11.825,0.0,20.0,966.105041,103.393158,1.0,1.81,7017.1875,6261.596654,6250.284180
2,2020-03-10T05:00:00.215Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,119.234494,0.0003,...,-11.825,0.0,20.0,966.105041,103.393158,1.0,1.81,6344.6875,5105.590799,5093.851074
3,2020-03-10T05:00:00.281Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,142.959518,0.0003,...,-11.825,0.0,20.0,966.105041,103.393158,1.0,1.81,5192.1875,3989.562972,3978.289795
4,2020-03-10T05:00:00.348Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,133.534509,0.0003,...,-11.825,0.0,20.0,966.105041,103.393158,1.0,1.81,4064.6875,2907.409656,2896.432617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295708,2020-03-11T04:59:59.703Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,105.259480,0.0003,...,-11.825,0.0,20.0,966.085022,103.419640,1.0,1.81,4063.4375,2907.409656,2896.377441
1295709,2020-03-11T04:59:59.770Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,103.959479,0.0003,...,-11.825,0.0,20.0,966.085022,103.419640,1.0,1.81,2977.1875,1855.773918,1845.101929
1295710,2020-03-11T04:59:59.837Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,92.259467,0.0003,...,-11.825,0.0,20.0,966.085022,103.419640,1.0,1.81,1924.6875,833.740231,823.275757
1295711,2020-03-11T04:59:59.903Z,MLParamData_1583906408.4261804_From_MLrn_2020-...,0.753501,10.000006,0.3,3.0,0.0,0.0,105.259480,0.0003,...,-11.825,0.0,20.0,966.085022,103.419640,1.0,1.81,894.6875,369.415282,370.404053


In [4]:
# Need to shift this back because we use the previous time step's value
boostr['B:VIMIN'] = boostr['B:VIMIN'].shift(-1)

In [5]:
X_unscaled = boostr.loc[:249999,['B:VIMIN', 'B:IMINER', 'B_VIMIN', 'B:LINFRQ', 'I:IB', 'I:MDAT40']].to_numpy(copy=True, dtype=np.float32)

In [6]:
# Scale the input values to roughly mean of one, centered at zero. 

X = np.zeros(X_unscaled.shape)
for i in range(X_unscaled.shape[1]):
    data = X_unscaled[:,i]
    median = np.median(data)
    upper_quartile = np.percentile(data, 75)
    lower_quartile = np.percentile(data, 25)
    iqr = upper_quartile - lower_quartile
    lower_whisker = data[data>=lower_quartile-1.5*iqr].min()
    upper_whisker = data[data<=upper_quartile+1.5*iqr].max()
    ranged = upper_whisker - lower_whisker
    print(f"var {i} with {median=}, {upper_whisker=}, {lower_whisker=}")
    X[:,i] = 1/ranged * (data - median)

var 0 with median=103.39773, upper_whisker=103.45586, lower_whisker=103.33849
var 1 with median=-0.0003125, upper_whisker=0.5196875, lower_whisker=-0.5203125
var 2 with median=103.39916, upper_whisker=103.401665, lower_whisker=103.39316
var 3 with median=7.433795, upper_whisker=37.331104, lower_whisker=-31.444073
var 4 with median=2975.9375, upper_whisker=7060.9375, lower_whisker=345.9375
var 5 with median=2907.4097, upper_whisker=7033.386, lower_whisker=369.41528


In [7]:
X_unscaled[:5,:]

array([[ 1.03421227e+02,  1.34687498e-01,  1.03393158e+02,
         2.61179085e+01,  7.05593750e+03,  7.03323340e+03],
       [ 1.03425453e+02, -2.70312488e-01,  1.03393158e+02,
         2.60633430e+01,  7.01718750e+03,  6.26159668e+03],
       [ 1.03420357e+02, -3.05312514e-01,  1.03393158e+02,
         2.60087776e+01,  6.34468750e+03,  5.10559082e+03],
       [ 1.03412270e+02, -2.60312498e-01,  1.03393158e+02,
         2.60815315e+01,  5.19218750e+03,  3.98956299e+03],
       [ 1.03403305e+02, -1.85312495e-01,  1.03393158e+02,
         2.62088509e+01,  4.06468750e+03,  2.90740967e+03]], dtype=float32)

In [8]:
X[:5,:]

array([[ 0.20022102,  0.1298077 , -0.70582956,  0.27166942,  0.60759497,
         0.61912394],
       [ 0.2362348 , -0.25961536, -0.70582956,  0.27087602,  0.60182428,
         0.5033316 ],
       [ 0.19281024, -0.29326925, -0.70582956,  0.27008262,  0.50167537,
         0.32986057],
       [ 0.12390301, -0.25      , -0.70582956,  0.27114049,  0.33004469,
         0.16238867],
       [ 0.04751999, -0.17788461, -0.70582956,  0.27299172,  0.162137  ,
         0.        ]])

In [9]:
from tensorflow import keras

MODEL = "policy_model_e940/policy_model_e940_.modelall.h5"
model = keras.models.load_model(MODEL)

#WEIGHTS = "MLP_model/final_policy_model.weights.h5"
#DESC = "MLP_model/final_policy_model.json"

#WEIGHTS = "untrained_model/initialized_weights.h5"
#DESC = "untrained_model/untrained_model.json"

#with open(DESC) as fl:
#    desc = "\n".join(fl.readlines())
#model = keras.models.model_from_json(desc)
#model.load_weights(WEIGHTS)


2021-10-19 16:35:51.701228: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               896       
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 903       
Total params: 34,823
Trainable params: 34,823
Non-trainable params: 0
_________________________________________________________________


In [11]:
Y = model.predict(X)

2021-10-19 16:35:51.848852: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [12]:
len(Y)


250000

In [13]:
Y[:5,:]

array([[-0.5329083 , -0.68004626, -0.7054163 , -0.58849317, -0.61750364,
        -0.49551165, -0.6340498 ],
       [-1.2135415 , -1.5318651 , -1.2240001 , -1.284127  , -1.3369267 ,
        -1.489888  , -1.3130816 ],
       [-1.1527396 , -1.4675287 , -1.1790047 , -1.2274864 , -1.2677    ,
        -1.428726  , -1.2455697 ],
       [-0.9002469 , -1.1200418 , -0.9792087 , -0.9466856 , -1.0155969 ,
        -0.9784032 , -1.0155427 ],
       [-0.5888474 , -0.72858685, -0.725708  , -0.625678  , -0.6707119 ,
        -0.556134  , -0.6852273 ]], dtype=float32)

In [14]:
import hls4ml

config = hls4ml.utils.config_from_keras_model(model, granularity='model')
print("-----------------------------------")
print("Configuration")
print(config)
print("-----------------------------------")

Interpreting Model
Topology:
Layer name: input_2, layer type: Input
Layer name: dense_4, layer type: Dense
  -> Activation (relu), layer name: dense_4
Layer name: dense_5, layer type: Dense
  -> Activation (relu), layer name: dense_5
Layer name: dense_6, layer type: Dense
  -> Activation (relu), layer name: dense_6
Layer name: dense_7, layer type: Dense
  -> Activation (linear), layer name: dense_7
-----------------------------------
Configuration
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}}
-----------------------------------


In [15]:
config["Model"]["ReuseFactor"] = 128
config["Model"]["Precision"] = 'ac_fixed<20,6>'

print("-----------------------------------")
print("Configuration")
print(config)
print("-----------------------------------")


-----------------------------------
Configuration
{'Model': {'Precision': 'ac_fixed<20,6>', 'ReuseFactor': 128, 'Strategy': 'Latency'}}
-----------------------------------


In [16]:

hls_model = hls4ml.converters.convert_from_keras_model(model, backend="Quartus",
                                                       hls_config=config,
                                                       output_dir='policy_prj_e940_external',
                                                       part='Arria10',
                                                       external_weights=True,
                                                       project_name='policy')

Interpreting Model
Topology:
Layer name: input_2, layer type: InputLayer, input shapes: [[None, 6]], output shape: [None, 6]
Layer name: dense_4, layer type: Dense, input shapes: [[None, 6]], output shape: [None, 128]
Layer name: dense_5, layer type: Dense, input shapes: [[None, 128]], output shape: [None, 128]
Layer name: dense_6, layer type: Dense, input shapes: [[None, 128]], output shape: [None, 128]
Layer name: dense_7, layer type: Dense, input shapes: [[None, 128]], output shape: [None, 7]
Creating HLS model


In [17]:
hls_model.compile()

Writing HLS project
Done


In [18]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True)

In [19]:
Y_hls = hls_model.predict(np.ascontiguousarray(X))

In [20]:
Y_hls[:5,:]

array([[-0.53344727, -0.68066406, -0.70581055, -0.58905029, -0.61816406,
        -0.49609375, -0.63464355],
       [-1.21337891, -1.53167725, -1.22387695, -1.28399658, -1.33666992,
        -1.48974609, -1.31280518],
       [-1.15283203, -1.46746826, -1.17907715, -1.22747803, -1.26763916,
        -1.42895508, -1.24542236],
       [-0.90026855, -1.12005615, -0.97918701, -0.94689941, -1.01556396,
        -0.9786377 , -1.01544189],
       [-0.58868408, -0.72839355, -0.72558594, -0.62554932, -0.67047119,
        -0.5559082 , -0.68499756]])

In [21]:
diff = Y - Y_hls

In [22]:
diff[:10,:]

array([[ 5.38945198e-04,  6.17802143e-04,  3.94225121e-04,
         5.57124615e-04,  6.60419464e-04,  5.82098961e-04,
         5.93781471e-04],
       [-1.62601471e-04, -1.87873840e-04, -1.23143196e-04,
        -1.30414963e-04, -2.56776810e-04, -1.41859055e-04,
        -2.76446342e-04],
       [ 9.23871994e-05, -6.04391098e-05,  7.24792480e-05,
        -8.34465027e-06, -6.07967377e-05,  2.29120255e-04,
        -1.47342682e-04],
       [ 2.16364861e-05,  1.43051147e-05, -2.16960907e-05,
         2.13801861e-04, -3.29017639e-05,  2.34484673e-04,
        -1.00851059e-04],
       [-1.63316727e-04, -1.93297863e-04, -1.22070312e-04,
        -1.28686428e-04, -2.40683556e-04, -2.25782394e-04,
        -2.29716301e-04],
       [ 2.84314156e-04,  3.09884548e-04,  2.18242407e-04,
         2.88367271e-04,  2.68578529e-04,  3.08275223e-04,
         3.09705734e-04],
       [ 2.26169825e-04,  1.89244747e-04,  1.89065933e-04,
         2.26944685e-04,  2.01821327e-04,  2.87055969e-04,
         1.8343329

In [23]:
mapping = [0, 0.0001, 0.005, 0.001, -0.0001, -0.005, -0.001]

In [24]:
Y_mapped = np.argmax(Y, axis=1)

In [25]:
Y_hls_mapped = np.argmax(Y_hls, axis=1)

In [26]:
numwrong = np.count_nonzero(Y_mapped - Y_hls_mapped)

In [27]:
numwrong

417

In [28]:
numwrong / len(Y_mapped)

0.001668

In [29]:
hls_model.build()

Exception: Intel HLS installation not found. Make sure "i++" is on PATH.

In [ ]:
np.savetxt("Y_hls.txt", Y_hls, delimiter=" ")

In [ ]:
np.savetxt("X.txt", X, delimiter=" ")